In [1]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

### Step 0: Load Libraries and Data

In [3]:
df = spark.read.format("csv") \
  .option("inferSchema", "true").option("header", "true") \
  .load("s3a://datapalooza/airbnb/airbnb.csv.bz2")

df.registerTempTable("df")

print(df.head())

Row(id=5731498, name='A 2-bdrm house in Plaka of Athens', space='Ideally located a unique house in a very peaceful neighborhood of Plaka, near Acropolis. It is a traditional house in the heart of the historical center of Athens, in Plaka. The kitchen is fully equipped with oven, fridge with freezer. Cutlery, dishes and pans, kettle, espresso coffee maker (espresso capsules are provided), toaster. There is also a vacuum cleaner and a laundry machine. One big closet will make your stay more comfortable. Bed linen, towels and bath amenities are provided. Moreover, the apartment is fully airconditioned. The apartment is very close to a greek traditional tavernas, a pharmacy, banks and public transport.  Airport or any other transport is available upon demand at an additional but very reasonable cost. ', price='120.0', bathrooms='1.0', bedrooms='2.0', room_type='Entire home/apt', square_feet=None, host_is_super_host='0.0', city='Athina', state=None, cancellation_policy='moderate', security_

In [4]:
print(df.count())

198454


### Step 1: Clean, Filter, and Summarize the Data

In [5]:
df_filtered = df.filter("price >= 50 AND price <= 750 AND bathrooms > 0.0 AND bedrooms is not null")

df_filtered.registerTempTable("df_filtered")

df_final = spark.sql("""
    select
        id,
        city,
        case when state in('NY', 'CA', 'London', 'Berlin', 'TX' ,'IL', 'OR', 'DC', 'WA')
            then state
            else 'Other'
        end as state,
        space,
        cast(price as double) as price,
        cast(bathrooms as double) as bathrooms,
        cast(bedrooms as double) as bedrooms,
        room_type,
        host_is_super_host,
        cancellation_policy,
        cast(case when security_deposit is null
            then 0.0
            else security_deposit
        end as double) as security_deposit,
        price_per_bedroom,
        cast(case when number_of_reviews is null
            then 0.0
            else number_of_reviews
        end as double) as number_of_reviews,
        cast(case when extra_people is null
            then 0.0
            else extra_people
        end as double) as extra_people,
        instant_bookable,
        cast(case when cleaning_fee is null
            then 0.0
            else cleaning_fee
        end as double) as cleaning_fee,
        cast(case when review_scores_rating is null
            then 80.0
            else review_scores_rating
        end as double) as review_scores_rating,
        cast(case when square_feet is not null and square_feet > 100
            then square_feet
            when (square_feet is null or square_feet <=100) and (bedrooms is null or bedrooms = 0)
            then 350.0
            else 380 * bedrooms
        end as double) as square_feet
    from df_filtered
""").persist()

df_final.registerTempTable("df_final")

df_final.select("square_feet", "price", "bedrooms", "bathrooms", "cleaning_fee").describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+
|summary|       square_feet|             price|          bedrooms|         bathrooms|     cleaning_fee|
+-------+------------------+------------------+------------------+------------------+-----------------+
|  count|            151770|            151770|            151770|            151770|           151770|
|   mean| 545.5359161889702|130.99831982605258|    1.333537589774|1.1987316333926337|37.24688014759175|
| stddev|363.18878116083886| 89.57902021660226|0.8457954996720247|0.4834034060595014|  42.621395309755|
|    min|             104.0|              50.0|               0.0|               0.5|              0.0|
|    max|           32292.0|             750.0|              10.0|               8.0|            700.0|
+-------+------------------+------------------+------------------+------------------+-----------------+



In [6]:
print(df_final.count())

151770


In [7]:
print(df_final.schema)

StructType(List(StructField(id,IntegerType,true),StructField(city,StringType,true),StructField(state,StringType,true),StructField(space,StringType,true),StructField(price,DoubleType,true),StructField(bathrooms,DoubleType,true),StructField(bedrooms,DoubleType,true),StructField(room_type,StringType,true),StructField(host_is_super_host,StringType,true),StructField(cancellation_policy,StringType,true),StructField(security_deposit,DoubleType,true),StructField(price_per_bedroom,StringType,true),StructField(number_of_reviews,DoubleType,true),StructField(extra_people,DoubleType,true),StructField(instant_bookable,StringType,true),StructField(cleaning_fee,DoubleType,true),StructField(review_scores_rating,DoubleType,true),StructField(square_feet,DoubleType,true)))


In [8]:
# Most popular cities

spark.sql("""
    select 
        state,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by state
    order by count(*) desc
""").show()

+------+-----+------------------+---------+
| state|   ct|         avg_price|max_price|
+------+-----+------------------+---------+
| Other|87420|122.00251658659346|    750.0|
|    NY|22887|145.93550924105386|    750.0|
|    CA|20726|157.41102962462608|    750.0|
|Berlin| 6031| 80.64118719946941|    650.0|
|    IL| 3552|141.46903153153153|    690.0|
|    TX| 3104|195.02673969072166|    750.0|
|    WA| 2698|131.48999258710157|    750.0|
|    DC| 2590|136.64015444015445|    720.0|
|    OR| 1954|114.02661207778915|    700.0|
|London|  808|108.85396039603961|    600.0|
+------+-----+------------------+---------+



In [9]:
# Most expensive popular cities

spark.sql("""
    select 
        city,
        count(*) as ct,
        avg(price) as avg_price,
        max(price) as max_price
    from df_final
    group by city
    order by avg(price) desc
""").filter("ct > 25").show()

+-------------------+---+------------------+---------+
|               city| ct|         avg_price|max_price|
+-------------------+---+------------------+---------+
|         Palm Beach| 26| 348.7692307692308|    701.0|
|        Watsonville| 38| 313.3157894736842|    670.0|
|             Malibu|136| 280.9852941176471|    750.0|
|             Avalon| 38|262.42105263157896|    701.0|
|           Capitola| 35|             246.4|    650.0|
|           Tamarama| 72|             238.5|    750.0|
|    Manhattan Beach|109|232.10091743119267|    700.0|
|Rancho Palos Verdes| 39|230.02564102564102|    750.0|
|       Avalon Beach| 38|229.60526315789474|    620.0|
|            Newport| 52| 223.8653846153846|    750.0|
|      Darling Point| 29|221.51724137931035|    623.0|
|        Middle Park| 34| 212.7941176470588|    671.0|
|            Balmain| 55|212.56363636363636|    712.0|
|        North Bondi|180|206.68333333333334|    750.0|
|             Bronte|144|203.70833333333334|    750.0|
|        Q

### Step 2: Define Continous and Categorical Features


In [10]:
continuous_features = ["bathrooms", \
                       "bedrooms", \
                       "security_deposit", \
                       "cleaning_fee", \
                       "extra_people", \
                       "number_of_reviews", \
                       "square_feet", \
                       "review_scores_rating"]

categorical_features = ["room_type", \
                        "host_is_super_host", \
                        "cancellation_policy", \
                        "instant_bookable", \
                        "state"]

### Step 3: Split Data into Training and Validation 

In [11]:
[training_dataset, validation_dataset] = df_final.randomSplit([0.8, 0.2])

### Step 4: Continous Feature Pipeline

In [12]:
continuous_feature_assembler = VectorAssembler(inputCols=continuous_features, outputCol="unscaled_continuous_features")

continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_continuous_features", \
                                           withStd=True, withMean=False)

### Step 5: Categorical Feature Pipeline

In [13]:
categorical_feature_indexers = [StringIndexer(inputCol=x, \
                                              outputCol="{}_index".format(x)) \
                                for x in categorical_features]

categorical_feature_one_hot_encoders = [OneHotEncoder(inputCol=x.getOutputCol(), \
                                                      outputCol="oh_encoder_{}".format(x.getOutputCol() )) \
                                        for x in categorical_feature_indexers]

### Step 6: Assemble our features and feature pipeline


In [14]:
feature_cols_lr = [x.getOutputCol() \
                   for x in categorical_feature_one_hot_encoders]
feature_cols_lr.append("scaled_continuous_features")

feature_assembler_lr = VectorAssembler(inputCols=feature_cols_lr, \
                                       outputCol="features_lr")

### Step 7: Train a Linear Regression Model

In [15]:
linear_regression = LinearRegression(featuresCol="features_lr", \
                                     labelCol="price", \
                                     predictionCol="price_prediction", \
                                     maxIter=10, \
                                     regParam=0.3, \
                                     elasticNetParam=0.8)

estimators_lr = \
  [continuous_feature_assembler, continuous_feature_scaler] \
  + categorical_feature_indexers + categorical_feature_one_hot_encoders \
  + [feature_assembler_lr] + [linear_regression]

pipeline = Pipeline(stages=estimators_lr)

pipeline_model = pipeline.fit(training_dataset)

print(pipeline_model)

PipelineModel_44d1a622b76440a9a666


### TODO: Step 8:  Validate Linear Regression Model

### Step 9:  Convert PipelineModel to PMML

In [16]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(spark, training_dataset, pipeline_model)

print(pmmlBytes.decode("utf-8"))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">
	<Header>
		<Application/>
		<Timestamp>2017-01-19T22:34:54Z</Timestamp>
	</Header>
	<DataDictionary>
		<DataField name="bathrooms" optype="continuous" dataType="double"/>
		<DataField name="bedrooms" optype="continuous" dataType="double"/>
		<DataField name="security_deposit" optype="continuous" dataType="double"/>
		<DataField name="cleaning_fee" optype="continuous" dataType="double"/>
		<DataField name="extra_people" optype="continuous" dataType="double"/>
		<DataField name="number_of_reviews" optype="continuous" dataType="double"/>
		<DataField name="square_feet" optype="continuous" dataType="double"/>
		<DataField name="review_scores_rating" optype="continuous" dataType="double"/>
		<DataField name="room_type" optype="categorical" dataType="string">
			<Value value="Entire home/apt"/>
			<Value value="Private room"/>
			<Value value="Shared room"/>
		</DataField>
		<Dat

### Deployment Option 1:  Mutable Model Deployment
Deploy New Model to Live, Running Model Server

In [17]:
import urllib.request

update_url = 'http://prediction-pmml-aws.demo.pipeline.io/update-pmml/pmml_airbnb'

update_headers = {}
update_headers['Content-type'] = 'application/xml'

req = urllib.request.Request(update_url, \
                             headers=update_headers, \
                             data=pmmlBytes)

resp = urllib.request.urlopen(req)

print(resp.status) # Should return Http Status 200 

200


In [18]:
import urllib.request

update_url = 'http://prediction-pmml-gcp.demo.pipeline.io/update-pmml/pmml_airbnb'

update_headers = {}
update_headers['Content-type'] = 'application/xml'

req = urllib.request.Request(update_url, \
                             headers=update_headers, \
                             data=pmmlBytes)

resp = urllib.request.urlopen(req)

print(resp.status) # Should return Http Status 200 

200


In [19]:
import urllib.parse
import json

evaluate_url = 'http://prediction-pmml-aws.demo.pipeline.io/evaluate-pmml/pmml_airbnb'

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'

input_params = '{"bathrooms":2.0, \
                 "bedrooms":2.0, \
                 "security_deposit":175.00, \
                 "cleaning_fee":25.0, \
                 "extra_people":1.0, \
                 "number_of_reviews": 2.0, \
                 "square_feet": 250.0, \
                 "review_scores_rating": 2.0, \
                 "room_type": "Entire home/apt", \
                 "host_is_super_host": "0.0", \
                 "cancellation_policy": "flexible", \
                 "instant_bookable": "1.0", \
                 "state": "CA"}' 
encoded_input_params = input_params.encode('utf-8')

req = urllib.request.Request(evaluate_url, \
                             headers=evaluate_headers, \
                             data=encoded_input_params)

resp = urllib.request.urlopen(req)

print(resp.read())

b'{"results":[{"result": "fallback"}]'


In [20]:
import urllib.parse
import json

evaluate_url = 'http://prediction-pmml-gcp.demo.pipeline.io/evaluate-pmml/pmml_airbnb'

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'

input_params = '{"bathrooms":2.0, \
                 "bedrooms":2.0, \
                 "security_deposit":175.00, \
                 "cleaning_fee":25.0, \
                 "extra_people":1.0, \
                 "number_of_reviews": 2.0, \
                 "square_feet": 250.0, \
                 "review_scores_rating": 2.0, \
                 "room_type": "Entire home/apt", \
                 "host_is_super_host": "0.0", \
                 "cancellation_policy": "flexible", \
                 "instant_bookable": "1.0", \
                 "state": "CA"}' 
encoded_input_params = input_params.encode('utf-8')

req = urllib.request.Request(evaluate_url, \
                             headers=evaluate_headers, \
                             data=encoded_input_params)

resp = urllib.request.urlopen(req)

print(resp.read())

b'{"results":[{"result": "fallback"}]'


### Deployment Option 2:  Immutable Model Deployment
Save Model to Disk

In [23]:
with open('/root/pipeline/prediction.ml/pmml/data/pmml_airbnb/pmml_airbnb.pmml', 'wb') as f:
  f.write(pmmlBytes)

!cat /root/pipeline/prediction.ml/pmml/data/pmml_airbnb/pmml_airbnb.pmml

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">
	<Header>
		<Application/>
		<Timestamp>2017-01-19T22:34:54Z</Timestamp>
	</Header>
	<DataDictionary>
		<DataField name="bathrooms" optype="continuous" dataType="double"/>
		<DataField name="bedrooms" optype="continuous" dataType="double"/>
		<DataField name="security_deposit" optype="continuous" dataType="double"/>
		<DataField name="cleaning_fee" optype="continuous" dataType="double"/>
		<DataField name="extra_people" optype="continuous" dataType="double"/>
		<DataField name="number_of_reviews" optype="continuous" dataType="double"/>
		<DataField name="square_feet" optype="continuous" dataType="double"/>
		<DataField name="review_scores_rating" optype="continuous" dataType="double"/>
		<DataField name="room_type" optype="categorical" dataType="string">
			<Value value="Entire home/apt"/>
			<Value value="Private room"/>
			<Value value="Shared room"/>
		</DataField>
		<Dat